In [1]:
import pandas as pd
import csv
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import gensim
from gensim import corpora, models
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from pprint import pprint

stemmer = SnowballStemmer("english") # the stopwords 
def remain_english(line):
    rule =re.compile("[^a-z^A-Z]")
    line = rule.sub('',line)
    return line
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v')) # remain problems

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(str(text)) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
#             result.append(lemmatize_stemming(token))
            result.append(remain_english(lemmatize_stemming((token))).lower()) # take out not english and return into verb, and lower it 
    return result

C:\Users\zmz22\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\zmz22\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
# read file into pandas
import pandas as pd

fileGP = r'C:\Users\zmz22\Downloads\GamesReview.csv'
fileApple = r'C:\Users\zmz22\Downloads\apple-games-reviews.csv'

df1 = pd.read_csv(fileGP)
df2 = pd.read_csv(fileApple)
# take the target subcate reviews (Apple and GP)
dfActionGP = df1[df1.appSubGener.str.contains('Action')] # the Action games from GP
dfArcadeGP = df1[df1.appSubGener.str.contains('Arcade')] # the Arcade games from GP
dfActionAp = df2[df2.genres.str.contains('Action')] # the Action games from Apple 
dfArcadeAp = df2[df2.genres.str.contains('Arcade')] # the Arcade games from Apple 


# Action Part

In [4]:
# analyse Action part
df1 = dfActionGP
df2 = dfActionAp
gpNum = dfActionGP.count()['text']
ApNum = dfActionAp.count()['text'] # text(reviews) number from apple
print('targe subcate reviews num of GP and Apple:',gpNum,ApNum)

sid = SentimentIntensityAnalyzer()
text_num = gpNum + ApNum
documentNeg=[]
documentPos=[]
inconsistantNum1 = 0 # inconsistant reviews number in low rating reviews
inconsistantNum2 = 0
for i in df1.score.index:
    if(df1.score[i]<3): # low rating reviews
#         print(sid.polarity_scores(df1.text[i])['neg'])
        if(sid.polarity_scores(str(df1.text[i]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df1.text[i])             

    if(df1.score[i]>3): # high rating reviews
        if(sid.polarity_scores(str(df1.text[i]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df1.text[i]) 
            
for j in df2.score.index:
    if(df2.score[j]<3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df2.text[j])             
    if(df2.score[j]>3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df2.text[j])      
                        
print('number of reviews text:',text_num)
print('number of neg:',len(documentNeg))
print('number of inconsistant neg reviews text(actually a pos reviews):',inconsistantNum1)
print('number of pos:',len(documentPos))
print('number of inconsistant pos reviews text(actually an neg reviews):',inconsistantNum2)


targe subcate reviews num of GP and Apple: 33537 382637
number of reviews text: 416174
number of neg: 87589
number of inconsistant neg reviews text(actually a pos reviews): 481
number of pos: 284598
number of inconsistant pos reviews text(actually an neg reviews): 841


In [5]:
# negtive LDA  reviews analyse
document = documentNeg
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)

# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
print('made doc_term_matrix')

Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

made doc_term_matrix
object num 87589
Fitting Time: 2536.417199611664
Topic: 0 
Words: 0.105*"game" + 0.054*"play" + 0.046*"crash" + 0.040*"time" + 0.027*"screen" + 0.023*"work" + 0.021*"load" + 0.020*"open" + 0.019*"start" + 0.015*"updat"
Topic: 1 
Words: 0.047*"kill" + 0.045*"shoot" + 0.030*"weapon" + 0.020*"gun" + 0.017*"zombi" + 0.016*"need" + 0.016*"mission" + 0.014*"like" + 0.013*"tank" + 0.011*"car"
Topic: 2 
Words: 0.071*"game" + 0.050*"money" + 0.035*"spend" + 0.020*"play" + 0.015*"upgrad" + 0.014*"time" + 0.010*"purchas" + 0.009*"want" + 0.009*"gold" + 0.008*"coin"
Topic: 3 
Words: 0.051*"connect" + 0.038*"server" + 0.034*"game" + 0.031*"work" + 0.029*"play" + 0.027*"say" + 0.021*"onlin" + 0.019*"internet" + 0.018*"wifi" + 0.018*"error"
Topic: 4 
Words: 0.100*"level" + 0.060*"game" + 0.057*"star" + 0.017*"imposs" + 0.017*"rat" + 0.017*"challeng" + 0.015*"beat" + 0.014*"score" + 0.013*"rate" + 0.013*"time"
Topic: 5 
Words: 0.042*"player" + 0.042*"game" + 0.024*"play" + 0.019*"

In [6]:
# positive LDA reviews analyse
document = documentPos
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)
# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

object num 284598
Fitting Time: 8687.176492452621
Topic: 0 
Words: 0.084*"game" + 0.041*"great" + 0.037*"control" + 0.035*"graphic" + 0.027*"version" + 0.019*"gameplay" + 0.018*"iphon" + 0.014*"updat" + 0.014*"good" + 0.012*"work"
Topic: 1 
Words: 0.024*"like" + 0.021*"jump" + 0.017*"bird" + 0.014*"sound" + 0.013*"turn" + 0.012*"ball" + 0.012*"sonic" + 0.012*"button" + 0.010*"right" + 0.009*"music"
Topic: 2 
Words: 0.056*"game" + 0.025*"money" + 0.018*"time" + 0.016*"spend" + 0.014*"play" + 0.014*"upgrad" + 0.012*"free" + 0.010*"coin" + 0.010*"great" + 0.009*"good"
Topic: 3 
Words: 0.049*"like" + 0.030*"zombi" + 0.028*"shoot" + 0.025*"gun" + 0.024*"kill" + 0.023*"weapon" + 0.017*"need" + 0.011*"thing" + 0.010*"car" + 0.009*"think"
Topic: 4 
Words: 0.098*"level" + 0.055*"game" + 0.039*"challeng" + 0.027*"hard" + 0.027*"easi" + 0.025*"play" + 0.023*"beat" + 0.022*"time" + 0.017*"score" + 0.016*"addict"
Topic: 5 
Words: 0.188*"game" + 0.134*"love" + 0.058*"play" + 0.057*"great" + 0.051*"t

# The Arcade Part

In [7]:
# analyse Arcade part
df1 = dfArcadeGP
df2 = dfArcadeAp
gpNum = dfArcadeGP.count()['text']
ApNum = dfArcadeAp.count()['text'] # text(reviews) number from apple
print('targe subcate reviews num of GP and Apple:',gpNum,ApNum)

sid = SentimentIntensityAnalyzer()
text_num = gpNum + ApNum
documentNeg=[]
documentPos=[]
inconsistantNum1 = 0 # inconsistant reviews number in low rating reviews
inconsistantNum2 = 0
for i in df1.score.index:
    if(df1.score[i]<3): # low rating reviews
#         print(sid.polarity_scores(df1.text[i])['neg'])
        if(sid.polarity_scores(str(df1.text[i]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df1.text[i])             

    if(df1.score[i]>3): # high rating reviews
        if(sid.polarity_scores(str(df1.text[i]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df1.text[i]) 
            
for j in df2.score.index:
    if(df2.score[j]<3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df2.text[j])             
    if(df2.score[j]>3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df2.text[j])      
                        
print('number of reviews text:',text_num)
print('number of neg:',len(documentNeg))
print('number of inconsistant neg reviews text(actually a pos reviews):',inconsistantNum1)
print('number of pos:',len(documentPos))
print('number of inconsistant pos reviews text(actually an neg reviews):',inconsistantNum2)


targe subcate reviews num of GP and Apple: 12725 330051
number of reviews text: 342776
number of neg: 67945
number of inconsistant neg reviews text(actually a pos reviews): 419
number of pos: 238348
number of inconsistant pos reviews text(actually an neg reviews): 663


In [8]:
# negtive LDA  reviews analyse
document = documentNeg
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)

# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
print('made doc_term_matrix')

Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

made doc_term_matrix
object num 67945
Fitting Time: 1642.3280847072601
Topic: 0 
Words: 0.137*"game" + 0.036*"like" + 0.030*"play" + 0.029*"time" + 0.019*"stupid" + 0.018*"score" + 0.017*"thing" + 0.016*"wast" + 0.016*"hate" + 0.015*"bore"
Topic: 1 
Words: 0.079*"game" + 0.033*"like" + 0.031*"review" + 0.030*"version" + 0.027*"star" + 0.022*"updat" + 0.018*"play" + 0.017*"peopl" + 0.016*"think" + 0.014*"chang"
Topic: 2 
Words: 0.125*"game" + 0.055*"play" + 0.030*"time" + 0.029*"watch" + 0.023*"second" + 0.018*"add" + 0.012*"free" + 0.012*"minut" + 0.012*"like" + 0.011*"annoy"
Topic: 3 
Words: 0.040*"purchas" + 0.022*"support" + 0.020*"race" + 0.016*"spin" + 0.012*"issu" + 0.011*"develop" + 0.011*"buy" + 0.011*"custom" + 0.011*"account" + 0.010*"say"
Topic: 4 
Words: 0.174*"level" + 0.043*"game" + 0.027*"play" + 0.021*"time" + 0.020*"challeng" + 0.019*"pass" + 0.018*"imposs" + 0.017*"beat" + 0.012*"live" + 0.012*"like"
Topic: 5 
Words: 0.055*"mode" + 0.053*"connect" + 0.029*"tabl" + 0.0

In [9]:
# positive LDA reviews analyse
document = documentPos
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)
# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

object num 238348
Fitting Time: 4500.319480895996
Topic: 0 
Words: 0.136*"game" + 0.036*"good" + 0.032*"play" + 0.028*"updat" + 0.028*"best" + 0.023*"awesom" + 0.022*"like" + 0.018*"version" + 0.016*"work" + 0.015*"thank"
Topic: 1 
Words: 0.076*"like" + 0.070*"game" + 0.028*"think" + 0.026*"cool" + 0.026*"thing" + 0.023*"charact" + 0.020*"mode" + 0.019*"differ" + 0.016*"mayb" + 0.014*"add"
Topic: 2 
Words: 0.117*"game" + 0.059*"great" + 0.029*"graphic" + 0.027*"good" + 0.024*"time" + 0.020*"control" + 0.019*"simpl" + 0.017*"play" + 0.015*"nice" + 0.015*"gameplay"
Topic: 3 
Words: 0.031*"kill" + 0.031*"weapon" + 0.031*"shoot" + 0.028*"jump" + 0.022*"upgrad" + 0.020*"player" + 0.018*"team" + 0.018*"boss" + 0.014*"bomb" + 0.011*"enemi"
Topic: 4 
Words: 0.155*"game" + 0.117*"play" + 0.050*"addict" + 0.037*"love" + 0.031*"time" + 0.020*"like" + 0.018*"stop" + 0.015*"recommend" + 0.015*"great" + 0.015*"download"
Topic: 5 
Words: 0.171*"level" + 0.038*"game" + 0.033*"challeng" + 0.026*"hard" 